In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import plotly.express as px

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, plot, iplot

In [ ]:
current_data = pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv')
current_data.head()

In [ ]:
type(current_data)

In [ ]:
## plotting of confirmed cases

In [ ]:
# Choropleth Map of the World
fig = px.choropleth(current_data,locations='Country',locationmode='country names',color='Confirmed',animation_frame='Date')
fig.update_layout(title='Choropleth Map of Confirmed Cases -till today',template="plotly_dark")
fig.show()


In [ ]:
fig = px.choropleth(current_data,locations='Country',locationmode='country names',color='Confirmed',animation_frame='Date',scope='europe')
fig.update_layout(title='Choropleth Map of Confirmed Cases - Europe till today',template="plotly_dark")
fig.show()

In [ ]:
fig = px.choropleth(current_data,locations='Country',locationmode='country names',color='Confirmed',animation_frame='Date',scope='asia')
fig.update_layout(title='Choropleth Map of Confirmed Cases - Asia on 28-09-2020',template="plotly_dark")
fig.show()

In [ ]:
##### Spread over Time

In [ ]:
fig = px.scatter_geo(current_data,locations='Country',locationmode='country names',color='Confirmed',size='Confirmed',hover_name="Country",animation_frame='Date',title='Spread over Time')
fig.update(layout_coloraxis_showscale=False,layout_template="plotly_dark")
fig.show()

In [ ]:
#### The spread is very fast. It started in China and spread to the complete world

In [ ]:
#### plotting of Recovered cases

In [ ]:
fig = px.choropleth(current_data,locations='Country',locationmode='country names',color='Recovered',animation_frame='Date')
fig.update_layout(title='Choropleth Map of Recovered Cases -till today',template="plotly_dark")
fig.show()


In [ ]:
fig = px.scatter_geo(current_data,locations='Country',locationmode='country names',color='Recovered',size='Recovered',hover_name="Country",animation_frame='Date',title='Recovery over Time')
fig.update(layout_coloraxis_showscale=False,layout_template="plotly_dark")
fig.show()

In [ ]:
#### From above interactive map we can say that recovery is very slow as compared to the spread.

In [ ]:
##### plotting of Deaths

In [ ]:
fig = px.choropleth(current_data,locations='Country',locationmode='country names',color='Deaths',animation_frame='Date')
fig.update_layout(title='Choropleth Map of Deaths -till today',template="plotly_dark")
fig.show()

In [ ]:
worldometer = pd.read_csv(r'/Users/gowthammarrapu/Documents/untitled folder 2/Covid-19/worldometer_data.csv')
worldometer.head()

In [ ]:
#### highlighting maximum values

In [ ]:
worldometer.style.background_gradient(cmap='RdPu')

In [ ]:
#### function For Comparison that will compare different stuffs

In [ ]:
def plot(df,x,y,xaxis_label,yaxis_label,title):
    fig = px.bar(worldometer.head(10), y=y,x=x,color='WHO Region')
    fig.update_layout(title=title,xaxis_title=xaxis_label,yaxis_title=yaxis_label)
    fig.show()

In [ ]:
##### Comparison of Tests/Million of 10 Most Affected Countries

In [ ]:
plot(worldometer.head(10),'Country/Region','Tests/1M pop','Country','Tests/M pop','Comparison of Tests/Million of 10 Most Affected Countries')

In [ ]:
#### extract latitudes & longtidues of locations

In [ ]:
import geopy
from geopy.geocoders import Nominatim

In [ ]:
geolocator=Nominatim(user_agent="app")

In [ ]:
location = geolocator.geocode("USA")
print(location.latitude)

In [ ]:
location.longitude

In [ ]:
current_data.head()

In [ ]:
df=current_data.copy()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df[df['Country']=='Afghanistan']

In [ ]:
df2=df.groupby(['Country'])[['Confirmed','Recovered','Deaths']].max().reset_index()

In [ ]:
df2.head()

In [ ]:
df2[df2['Country']=='India']

In [ ]:
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in df2['Country']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

In [ ]:
lat_lon

In [ ]:
df2['geo_loc']=lat_lon

In [ ]:
#### unzip it
lat,lon=zip(*np.array(df2['geo_loc']))

In [ ]:
df2['lat']=lat
df2['lon']=lon

In [ ]:
df2.head()

In [ ]:
df2.drop(['geo_loc'],axis=1,inplace=True)

In [ ]:
df2.head()

In [ ]:
df2.to_csv('F:/Spatial Analysis/Spatial_data.csv')

In [ ]:
## places which cases are Confirmed recently through the world in the past day alone¶

In [ ]:
import folium

In [ ]:
folium.Map(tiles='openstreetmap', zoom_start=2)

In [ ]:
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

for id,row in df2.iterrows():
    folium.Marker(location=[row['lat'],row['lon']], popup=row['Confirmed']).add_to(m)

m

In [ ]:
##### These are places which cases are Confirmed recently through the world in the past day alone

In [ ]:
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

for idx, row in df2.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['Recovered']).add_to(m)

m

In [ ]:
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

for idx, row in df2.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['Deaths']).add_to(m)

m

In [ ]:
##### Deaths are from these marked placesin the past day alone

In [ ]:
m = folium.Map(location=[54,15], tiles='cartodbpositron', zoom_start=2)

from folium.plugins import MarkerCluster
mc = MarkerCluster()
for idx, row in df2.iterrows():
    mc.add_child(folium.Marker([row['lat'], row['lon']],popup=row['Confirmed']))
m.add_child(mc)

m

In [ ]:
##### These are the Total number cases registered till date in respective regions through out the world

In [ ]:
from folium.plugins import HeatMap

In [ ]:
df2.head()

In [ ]:
# Creating map with overall cases registered

In [ ]:
m = folium.Map(location=[54,15], zoom_start=2)
HeatMap(data=df2[['lat', 'lon','Confirmed']], radius=15).add_to(m)

m

In [ ]:
##### In these regions the effect of corona virus is more . Countries like Brazil,India & US are suffering a lot.